In [422]:
import sys
import torch

In [423]:
sys.path.append('/envs')
sys.path.append('/models')
import envs.viper as vpr
import models.models as models
import models.dqn as dqn

In [424]:
import importlib
importlib.reload(vpr); importlib.reload(models); importlib.reload(dqn)

<module 'models.dqn' from 'h:\\Coding Projects\\Python Projects\\Deep-XIV-Rotation\\models\\dqn.py'>

In [425]:
viper_env = vpr.Viper()

b123


In [426]:
viper_env.get_max_actions()

11

In [427]:
agent = dqn.DQN(viper_env)

Layer (type:depth-idx)                   Output Shape              Param #
DenseNet                                 [8, 11]                   --
├─Linear: 1-1                            [8, 2048]                 10,240
├─Linear: 1-2                            [8, 11]                   22,539
Total params: 32,779
Trainable params: 32,779
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.26
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.13
Estimated Total Size (MB): 0.26


In [428]:
torch.device('cpu')

device(type='cpu')

In [429]:
inputs = torch.rand(5, viper_env.get_state_shape())
print(inputs)
preds = agent.get_action(inputs)
print(preds)

tensor([[4.5232e-01, 5.9615e-01, 9.6137e-01, 9.4490e-01],
        [9.8826e-01, 9.3744e-01, 6.7590e-01, 1.7399e-01],
        [4.6119e-01, 7.3011e-01, 6.9128e-01, 6.3731e-01],
        [3.8936e-01, 5.1713e-01, 6.9452e-02, 8.4366e-01],
        [8.8656e-04, 6.6542e-01, 4.5783e-02, 3.8699e-01]])
tensor([8, 8, 8, 8, 8], dtype=torch.int32)
